In [1]:
from _fallback import prune_conformers_symmetry, prune_conformers
from hypermolecule_class import graphize
from cclib.io import ccread
import numpy as np
import os

# os.chdir('Resources/tri')
# data = ccread('TSCoDe_checkpoint_May_28_15-59.xyz')
# ids = [31, 35, 32]
# constraints = np.array([[6, 66], [56, 71], [23, 34]])
# c_tuple = ((6, 66), (56, 71), (23, 34))

os.chdir('Resources/antara')
data = ccread('TSCoDe_TSs_guesses_May_31_16-56.xyz')
ids = [8, 8]
constraints = np.array([[0, 10], [2, 8]])
c_tuple = ((0, 10), (2, 8))

graph = graphize(data.atomcoords[0], data.atomnos)

In [2]:
edges = list(graph.edges)
for a, b in list(graph.edges):
    if a == b:
        edges.remove((a,b))
    elif (a, b) in c_tuple:
        edges.remove((a,b))
# edges

In [3]:
adj_mat = np.zeros((len(data.atomnos), len(data.atomnos)), dtype=int)
for x in range(len(adj_mat)):
    for y in range(len(adj_mat)):
        if tuple(sorted([x, y])) in edges:
            adj_mat[x, y] = 1
adj_mat

array([[0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]])

In [4]:
%%time
mask = prune_conformers_symmetry(data.atomcoords, data.atomnos, adj_mat, k=1, max_rmsd=0.5)[1]
print(f'kept {len([i for i in mask if i == True])}/{len(mask)}')

kept 12/24
Wall time: 8.2 s


This thing is tremendously slow. Maybe partitioning the matrix?

In [5]:
%%time
structures = data.atomcoords[:]
for k in (5000, 2000, 1000, 500, 200, 100, 50, 20, 10, 5, 2, 1):
    if 5*k < len(structures):
        structures, mask = prune_conformers_symmetry(structures, data.atomnos, adj_mat, max_rmsd=0.5, k=k)

print(f'kept {len([i for i in mask if i == True])}/{len(data.atomcoords)}')

kept 12/24
Wall time: 13.2 s


In [7]:
%%time
structures = data.atomcoords[:]
for k in (5000, 2000, 1000, 500, 200, 100, 50, 20, 10, 5, 2, 1):
    if 5*k < len(structures):
        structures, mask = prune_conformers(structures, max_rmsd=0.5, k=k)

print(f'kept {len([i for i in mask if i == True])}/{len(data.atomcoords)}')

kept 24/24
Wall time: 55 ms


In [8]:
13.2/(55/1000)

240.0